# 策略回测教程

本教程介绍 open-xquant 的核心管道：**Universe → Indicator → Signal → Rule** 四阶段模型。

我们将用一个 SMA 均线交叉策略（SMA10 金叉/死叉 SMA50）作为贯穿全文的示例，逐层拆解每个阶段的工作方式：

- **Indicator** — 路径无关的纯函数计算（向量化）
- **Signal** — 跨 symbol 截面操作，生成方向性预测（向量化）
- **Rule** — 路径相关的状态机，读取持仓生成订单（逐 bar）
- **BacktestEngine** — 串联四阶段，模拟交易，输出绩效

三个可替换 Protocol（MarketDataProvider、OrderRouter、FillReceiver）将策略定义与执行环境解耦——同一套策略代码，不改一行，可在回测、模拟盘、实盘之间切换。

## 1. 安装依赖

回测引擎是 open-xquant 核心功能，无需额外依赖。如需下载真实行情数据：

```bash
pip install open-xquant[yfinance]
```

---
## 2. Indicator — 技术指标

Indicator 是路径无关的纯函数：输入一个 symbol 的 DataFrame，输出等长 Series，引擎负责追加为宽表的新列。

SMA（简单移动平均线）是最基础的趋势指标：

In [ ]:
import pandas as pd
from oxq.indicators import SMA

# 构造一段模拟行情
dates = pd.bdate_range("2024-01-01", periods=10)
mktdata = pd.DataFrame({
    "close": [100, 102, 101, 105, 108, 107, 110, 112, 109, 115],
}, index=dates)

sma = SMA()
result = sma.compute(mktdata, period=3)

mktdata["sma_3"] = result
print("SMA(3) 计算结果：")
print(mktdata[["close", "sma_3"]])

前 `period - 1` 行是 NaN（滚动窗口不足），这是正常行为。Signal 和 Rule 层会处理这些 NaN。

**关键特性**：
- `compute` 是纯函数 — 不修改输入 DataFrame，不依赖外部状态
- 同一个 SMA 类可以用不同参数注册为多个实例（如 `sma_10` 和 `sma_50`）
- 默认对 `close` 列计算，也可以通过 `column` 参数指定其他列

验证 SMA 满足 Indicator Protocol：

In [ ]:
from oxq.core import Indicator

print(f"SMA 满足 Indicator Protocol: {isinstance(SMA(), Indicator)}")
print(f"Indicator name: {SMA().name}")

---
## 3. Signal — 信号生成

Signal 描述"交易的欲望"——方向性预测，而非交易指令。与 Indicator 的关键区别：

| 维度 | Indicator | Signal |
|------|-----------|--------|
| 输入 | 单个 symbol 的 DataFrame | **全 universe** 的 mktdata |
| 输出 | 一个 Series | **每个 symbol** 各一个 Series |
| 视角 | per symbol | cross-sectional（截面） |

Crossover 信号检测快线上穿慢线的时刻：

In [ ]:
from oxq.signals import Crossover

# 构造一组含有金叉的数据
dates = pd.bdate_range("2024-01-01", periods=6)
df = pd.DataFrame({
    "close": [100, 98, 97, 99, 102, 105],
    "sma_10": [99, 98, 97, 99, 101, 103],    # 快线
    "sma_50": [100, 100, 100, 100, 100, 100],  # 慢线
}, index=dates)

# Signal 接收整个 mktdata（dict），返回每个 symbol 的信号
mktdata_dict = {"AAPL": df}
crossover = Crossover()
signals = crossover.compute(mktdata_dict, fast="sma_10", slow="sma_50")

df["sma_10_x_sma_50"] = signals["AAPL"]
print("Crossover 信号：")
print(df[["sma_10", "sma_50", "sma_10_x_sma_50"]])

Day 4（2024-01-04）触发了上穿信号：前一天 sma_10(99) <= sma_50(100)，当天 sma_10(101) > sma_50(100)。

**Signal 为什么要接收全 universe？** 因为有些信号需要跨 symbol 操作——比如"按动量排名取 top 5"。Crossover 恰好只看单个 symbol，但 Protocol 为截面操作预留了能力。

---
## 4. Rule — 交易规则

Rule 是路径相关的——它知道当前持仓和资金状态，逐 bar 执行。

与 Indicator/Signal 的关键区别：

| 维度 | Indicator / Signal | Rule |
|------|-------------------|------|
| 输入 | 整个时间序列 | **单行**（当前 bar） |
| 状态 | 无状态（纯函数） | **有状态**（读取 Portfolio） |
| 输出 | Series | **Order \| None** |
| 计算模式 | 向量化 | 逐 bar 循环 |

In [ ]:
from oxq.core import Portfolio, Position
from oxq.rules import EntryRule, ExitRule

# EntryRule: 信号触发 + 无持仓 → 买入
entry = EntryRule(signal="sma_10_x_sma_50", shares=100)

# 模拟一个 bar 的数据
row = pd.Series({"close": 102.0, "sma_10": 101.0, "sma_50": 100.0, "sma_10_x_sma_50": True})
portfolio = Portfolio(cash=100_000.0)

order = entry.evaluate("AAPL", row, portfolio)
print(f"EntryRule 产生订单: {order}")

In [ ]:
# ExitRule: 快线 < 慢线 + 有持仓 → 卖出
exit_rule = ExitRule(fast="sma_10", slow="sma_50")

row_exit = pd.Series({"close": 95.0, "sma_10": 97.0, "sma_50": 100.0})
portfolio_with_pos = Portfolio(
    cash=50_000.0,
    positions={"AAPL": Position(symbol="AAPL", shares=100, avg_cost=102.0)},
)

sell_order = exit_rule.evaluate("AAPL", row_exit, portfolio_with_pos)
print(f"ExitRule 产生订单: {sell_order}")

**执行优先级**：引擎每个 bar 先执行 ExitRule（平仓），再执行 EntryRule（开仓）。避免同一个 bar 既买又卖。

---
## 5. Strategy — 声明式策略定义

Strategy 将 Universe、Indicator、Signal、Rule 组合为一个完整的声明式管道：

```
Universe     → 确定标的池
  ↓
Indicator    → 计算指标列（sma_10, sma_50）
  ↓
Signal       → 生成信号列（sma_10_x_sma_50）
  ↓
Rule         → 读取信号 + 持仓 → 生成订单
```

In [ ]:
from oxq.core import Strategy
from oxq.universe import StaticUniverse
from oxq.indicators import SMA
from oxq.signals import Crossover
from oxq.rules import EntryRule, ExitRule

strategy = Strategy(
    name="sma_crossover",
    hypothesis="短期均线上穿长期均线的标的在后续持有期内有正超额收益",
    universe=StaticUniverse(("AAPL",)),
    indicators={
        "sma_10": (SMA(), {"period": 10}),
        "sma_50": (SMA(), {"period": 50}),
    },
    signals={
        "sma_10_x_sma_50": (Crossover(), {"fast": "sma_10", "slow": "sma_50"}),
    },
    entry_rules=[EntryRule(signal="sma_10_x_sma_50", shares=100)],
    exit_rules=[ExitRule(fast="sma_10", slow="sma_50")],
)

print(f"策略名称: {strategy.name}")
print(f"假设: {strategy.hypothesis}")
print(f"指标: {list(strategy.indicators.keys())}")
print(f"信号: {list(strategy.signals.keys())}")
print(f"入场规则: {len(strategy.entry_rules)} 条")
print(f"出场规则: {len(strategy.exit_rules)} 条")

策略定义是纯声明式的——它描述"做什么"，不关心"怎么做"。同一个 Strategy 对象可以在回测、模拟盘、实盘中执行，代码零修改。

---
## 6. 宽表数据模型

在运行回测之前，先理解引擎如何处理数据。`mktdata` 是按 symbol 索引的 DataFrame 集合，各阶段通过**追加列**逐步加宽每个 symbol 的宽表：

```
原始行情              Indicator 后            Signal 后
+-----------+       +---------------+      +------------------+
| open      |       | open          |      | open             |
| high      |       | high          |      | high             |
| low       | ───►  | low           | ───► | low              |
| close     |       | close         |      | close            |
| volume    |       | volume        |      | volume           |
|           |       | sma_10  (新增)|      | sma_10           |
|           |       | sma_50  (新增)|      | sma_50           |
|           |       |               |      | sma_10_x_sma_50  |
+-----------+       +---------------+      +------------------+
```

这种设计的好处：Signal 无需知道 Indicator 的输出格式，只需按列名引用；Rule 同理。所有中间结果在同一张表上可见可查。

---
## 7. 运行回测

BacktestEngine 将四阶段串联执行。我们先下载真实数据，再运行完整回测。

In [ ]:
from oxq.data import YFinanceDownloader

# 下载 AAPL 2023-2024 两年数据
downloader = YFinanceDownloader()
path = downloader.download("AAPL", start="2023-01-01", end="2024-12-31")
print(f"数据已保存到: {path}")

In [ ]:
from oxq.backtest import BacktestEngine, SimBroker
from oxq.data import LocalMarketDataProvider

engine = BacktestEngine()
result = engine.run(
    strategy,
    market=LocalMarketDataProvider(),
    broker=SimBroker(),
    start="2023-01-01",
    end="2024-12-31",
)

print(f"总收益率:   {result.total_return():.2%}")
print(f"Sharpe Ratio: {result.sharpe_ratio():.2f}")
print(f"最大回撤:   {result.max_drawdown():.2%}")
print(f"交易次数:   {len(result.trades)}")

引擎内部执行流程：

1. **Phase 0 — Universe**：从 `StaticUniverse` 获取标的列表 `["AAPL"]`
2. **Phase 1 — Indicator**：对 AAPL 的 DataFrame 调用 `SMA.compute()`，追加 `sma_10`、`sma_50` 两列
3. **Phase 2 — Signal**：调用 `Crossover.compute(mktdata)`，追加 `sma_10_x_sma_50` 布尔列
4. **Phase 3 — Rule**：逐 bar 遍历，对每个 bar 先执行 ExitRule 再执行 EntryRule，通过 SimBroker 撮合订单

---
## 8. 查看交易记录

`result.trades` 包含所有成交记录（`Fill` 对象）：

In [ ]:
if result.trades:
    print(f"{'日期':<25} {'方向':>4}  {'数量':>4}  {'标的':<6} {'成交价':>8}")
    print("-" * 55)
    for fill in result.trades:
        print(
            f"{fill.filled_at:<25} {fill.order.side:>4}  "
            f"{fill.order.shares:>4}  {fill.order.symbol:<6} "
            f"{fill.filled_price:>8.2f}"
        )
else:
    print("无交易记录")

---
## 9. 查看宽表

`result.mktdata` 保留了完整的宽表，可以直接观察 Indicator 和 Signal 的计算结果：

In [ ]:
df = result.mktdata["AAPL"]
print(f"宽表列: {list(df.columns)}")
print(f"总行数: {len(df)}")
print()

# 显示信号触发点附近的数据
signal_days = df[df["sma_10_x_sma_50"] == True]
print(f"金叉触发次数: {len(signal_days)}")
if not signal_days.empty:
    print()
    print("金叉触发日的宽表数据：")
    print(signal_days[["close", "sma_10", "sma_50", "sma_10_x_sma_50"]])

---
## 10. 分阶段执行（Partial Execution）

引擎支持 `run_through` 参数，在任意阶段终止执行。这对逐组件独立评估非常有用——先验证 Indicator 是否合理，再看 Signal 是否有预测力，最后才加入 Rule 和仓位管理。

In [ ]:
# 只执行到 Indicator 阶段
result_ind = engine.run(
    strategy,
    market=LocalMarketDataProvider(),
    broker=SimBroker(),
    start="2023-01-01",
    end="2024-12-31",
    run_through="indicator",
)

df_ind = result_ind.mktdata["AAPL"]
print(f"Indicator 阶段 — 宽表列: {list(df_ind.columns)}")
print(f"交易次数: {len(result_ind.trades)}  (预期为 0)")
print()

# SMA 值预览
print(df_ind[["close", "sma_10", "sma_50"]].tail())

In [ ]:
# 只执行到 Signal 阶段
result_sig = engine.run(
    strategy,
    market=LocalMarketDataProvider(),
    broker=SimBroker(),
    start="2023-01-01",
    end="2024-12-31",
    run_through="signal",
)

df_sig = result_sig.mktdata["AAPL"]
print(f"Signal 阶段 — 宽表列: {list(df_sig.columns)}")
print(f"交易次数: {len(result_sig.trades)}  (预期为 0)")
print()

# 信号统计
n_signals = df_sig["sma_10_x_sma_50"].sum()
print(f"金叉信号总次数: {n_signals}")

---
## 11. 三接口架构

回测中的三个 Provider 角色：

| Protocol | 回测实现 | 说明 |
|----------|---------|------|
| `MarketDataProvider` | `LocalMarketDataProvider` | 从本地 Parquet 读取行情 |
| `OrderRouter` | `SimBroker` | 接收订单，队列管理 |
| `FillReceiver` | `SimBroker` | 模拟撮合，返回成交 |

SimBroker 同时实现了 OrderRouter 和 FillReceiver 两个 Protocol：

In [ ]:
from oxq.core import OrderRouter, FillReceiver
from oxq.backtest import SimBroker

broker = SimBroker()
print(f"SimBroker 满足 OrderRouter: {isinstance(broker, OrderRouter)}")
print(f"SimBroker 满足 FillReceiver: {isinstance(broker, FillReceiver)}")

未来切换到实盘时，只需替换 `SimBroker` 为 `LiveBroker`，策略代码不变：

```python
# 回测
engine.run(strategy, market=LocalMarketDataProvider(), broker=SimBroker(), ...)

# 实盘（未来）
engine.run(strategy, market=RealtimeData(), broker=LiveBroker(), ...)
```

---
## 12. 多标的策略

同一套策略定义，换一个 Universe 就变成多标的策略，引擎代码零修改：

In [ ]:
# 下载更多标的
for symbol in ["MSFT", "GOOGL"]:
    downloader.download(symbol, start="2023-01-01", end="2024-12-31")

# 只改 Universe，其余不变
multi_strategy = Strategy(
    name="sma_crossover_multi",
    hypothesis="短期均线上穿长期均线的标的在后续持有期内有正超额收益",
    universe=StaticUniverse(("AAPL", "MSFT", "GOOGL")),
    indicators={
        "sma_10": (SMA(), {"period": 10}),
        "sma_50": (SMA(), {"period": 50}),
    },
    signals={
        "sma_10_x_sma_50": (Crossover(), {"fast": "sma_10", "slow": "sma_50"}),
    },
    entry_rules=[EntryRule(signal="sma_10_x_sma_50", shares=100)],
    exit_rules=[ExitRule(fast="sma_10", slow="sma_50")],
)

result_multi = engine.run(
    multi_strategy,
    market=LocalMarketDataProvider(),
    broker=SimBroker(),
    start="2023-01-01",
    end="2024-12-31",
)

print(f"总收益率:   {result_multi.total_return():.2%}")
print(f"Sharpe Ratio: {result_multi.sharpe_ratio():.2f}")
print(f"最大回撤:   {result_multi.max_drawdown():.2%}")
print(f"交易次数:   {len(result_multi.trades)}")
print()

# 按标的分组显示交易
from collections import Counter
trade_counts = Counter(f.order.symbol for f in result_multi.trades)
for symbol, count in sorted(trade_counts.items()):
    print(f"  {symbol}: {count} 笔交易")

---
## 小结

本教程覆盖了策略回测管道的核心概念：

| 组件 | 职责 | 计算模式 |
|------|------|----------|
| `SMA` | 计算移动平均线 | 向量化，per symbol |
| `Crossover` | 检测上穿信号 | 向量化，cross-sectional |
| `EntryRule` | 信号触发时买入 | 逐 bar，有状态 |
| `ExitRule` | 快线跌破慢线时卖出 | 逐 bar，有状态 |
| `Strategy` | 声明式策略定义 | — |
| `BacktestEngine` | 执行四阶段管道 | — |
| `SimBroker` | 模拟撮合（OrderRouter + FillReceiver） | — |
| `BacktestResult` | 绩效指标 + 交易记录 + 宽表 | — |

**四阶段管道**：

```
Phase 0: Universe    → 确定标的池（StaticUniverse / FilterUniverse）
Phase 1: Indicator   → 向量化计算指标，追加为宽表新列
Phase 2: Signal      → 截面信号生成，追加为宽表新列
Phase 3: Rule        → 逐 bar 读取信号 + 持仓 → 生成订单 → SimBroker 撮合
```

**核心设计原则**：
- 策略定义与执行环境分离（三接口架构）
- Indicator/Signal 是纯函数，不修改 mktdata
- Rule 是状态机，读取持仓生成订单
- 宽表避免层间数据传递的复杂性
- `run_through` 支持逐组件独立评估